In [ ]:
#Installation of required libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score


from sklearn.model_selection import KFold
import warnings
warnings.simplefilter(action = "ignore")

In [ ]:
#Reading the dataset
df_train = pd.read_json("/content/train.json", orient='records', lines=True)
df_dev = pd.read_json("/content/dev.json", orient='records', lines=True)

xử lý missing value

In [ ]:
#số lần mang thai có thể =0 là hợp lí, nên k biến đổi nó về NaN
df_train[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df_train[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [ ]:
df_dev[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df_dev[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)


In [ ]:
df_temp = df_train

In [ ]:
df_train["Glucose"].fillna(df_temp["Glucose"].notnull().median(), inplace = True)
df_train["BloodPressure"].fillna(df_temp["BloodPressure"].notnull().median(), inplace = True)
df_train["SkinThickness"].fillna(df_temp["SkinThickness"].notnull().median(), inplace = True)
df_train["Insulin"].fillna(df_temp["Insulin"].notnull().median(), inplace = True)
df_train["BMI"].fillna(df_temp["BMI"].notnull().median(), inplace = True)

In [ ]:
df_dev["Glucose"].fillna(df_temp["Glucose"].notnull().median(), inplace = True)
df_dev["BloodPressure"].fillna(df_temp["BloodPressure"].notnull().median(), inplace = True)
df_dev["SkinThickness"].fillna(df_temp["SkinThickness"].notnull().median(), inplace = True)
df_dev["Insulin"].fillna(df_temp["Insulin"].notnull().median(), inplace = True)
df_dev["BMI"].fillna(df_temp["BMI"].notnull().median(), inplace = True)

xử lý outlier cho train

In [ ]:
# We determine outliers between all variables with the LOF method
from sklearn.neighbors import LocalOutlierFactor
lof =LocalOutlierFactor(n_neighbors= 10)
lof.fit_predict(df_train)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [ ]:
df_scores = lof.negative_outlier_factor_
np.sort(df_scores)[0:30]

array([-2.23250655, -2.11466237, -2.03765265, -2.01016783, -1.99830169,
       -1.79790245, -1.71480657, -1.67606288, -1.61991938, -1.56042657,
       -1.55897356, -1.53373688, -1.5157116 , -1.51098464, -1.50794333,
       -1.49840788, -1.49307669, -1.49027082, -1.46704163, -1.45091974,
       -1.43467896, -1.41776219, -1.41052339, -1.40944976, -1.40866866,
       -1.40506809, -1.40252256, -1.39702773, -1.39626372, -1.38888521])

In [ ]:
threshold = np.sort(df_scores)[7]
threshold

-1.6760628840090892

In [ ]:
#We delete those that are higher than the threshold
outlier = df_scores > threshold
df_train = df_train[outlier]

In [ ]:
# The size of the data set was examined.
df_train.shape

(529, 9)

feature enginearing trên train và dev

xử lý train

In [ ]:
# According to BMI, some ranges were determined and categorical variables were assigned.
NewBMI = pd.Series(["Underweight", "Normal", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"], dtype = "category")
df_train["NewBMI"] = NewBMI
df_train.loc[df_train["BMI"] < 18.5, "NewBMI"] = NewBMI[0]
df_train.loc[(df_train["BMI"] > 18.5) & (df_train["BMI"] <= 24.9), "NewBMI"] = NewBMI[1]
df_train.loc[(df_train["BMI"] > 24.9) & (df_train["BMI"] <= 29.9), "NewBMI"] = NewBMI[2]
df_train.loc[(df_train["BMI"] > 29.9) & (df_train["BMI"] <= 34.9), "NewBMI"] = NewBMI[3]
df_train.loc[(df_train["BMI"] > 34.9) & (df_train["BMI"] <= 39.9), "NewBMI"] = NewBMI[4]
df_train.loc[df_train["BMI"] > 39.9 ,"NewBMI"] = NewBMI[5]



In [ ]:
df_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI
0,1,80.0,55.0,1.0,0.0,19.1,0.258,21,0,Normal
1,1,124.0,74.0,36.0,0.0,27.8,0.100,30,0,Overweight
2,10,108.0,66.0,1.0,0.0,32.4,0.272,42,1,Obesity 1
3,2,120.0,76.0,37.0,105.0,39.7,0.215,29,0,Obesity 2
4,11,136.0,84.0,35.0,130.0,28.3,0.260,42,1,Overweight


In [ ]:
# A categorical variable creation process is performed according to the insulin value.
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"

In [ ]:
# The operation performed was added to the dataframe.
df_train = df_train.assign(NewInsulinScore=df_train.apply(set_insulin, axis=1))

In [ ]:
df_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI,NewInsulinScore
0,1,80.0,55.0,1.0,0.0,19.1,0.258,21,0,Normal,Abnormal
1,1,124.0,74.0,36.0,0.0,27.8,0.100,30,0,Overweight,Abnormal
2,10,108.0,66.0,1.0,0.0,32.4,0.272,42,1,Obesity 1,Abnormal
3,2,120.0,76.0,37.0,105.0,39.7,0.215,29,0,Obesity 2,Normal
4,11,136.0,84.0,35.0,130.0,28.3,0.260,42,1,Overweight,Normal


In [ ]:
# Some intervals were determined according to the glucose variable and these were assigned categorical variables.
NewGlucose = pd.Series(["Low", "Normal", "Overweight", "Secret", "High"], dtype = "category")
df_train["NewGlucose"] = NewGlucose
df_train.loc[df_train["Glucose"] <= 70, "NewGlucose"] = NewGlucose[0]
df_train.loc[(df_train["Glucose"] > 70) & (df_train["Glucose"] <= 99), "NewGlucose"] = NewGlucose[1]
df_train.loc[(df_train["Glucose"] > 99) & (df_train["Glucose"] <= 126), "NewGlucose"] = NewGlucose[2]
df_train.loc[df_train["Glucose"] > 126 ,"NewGlucose"] = NewGlucose[3]

In [ ]:
df_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI,NewInsulinScore,NewGlucose
0,1,80.0,55.0,1.0,0.0,19.1,0.258,21,0,Normal,Abnormal,Normal
1,1,124.0,74.0,36.0,0.0,27.8,0.100,30,0,Overweight,Abnormal,Overweight
2,10,108.0,66.0,1.0,0.0,32.4,0.272,42,1,Obesity 1,Abnormal,Overweight
3,2,120.0,76.0,37.0,105.0,39.7,0.215,29,0,Obesity 2,Normal,Overweight
4,11,136.0,84.0,35.0,130.0,28.3,0.260,42,1,Overweight,Normal,Secret


xử lý train dev

In [ ]:
# According to BMI, some ranges were determined and categorical variables were assigned.
NewBMI = pd.Series(["Underweight", "Normal", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"], dtype = "category")
df_dev["NewBMI"] = NewBMI
df_dev.loc[df_dev["BMI"] < 18.5, "NewBMI"] = NewBMI[0]
df_dev.loc[(df_dev["BMI"] > 18.5) & (df_dev["BMI"] <= 24.9), "NewBMI"] = NewBMI[1]
df_dev.loc[(df_dev["BMI"] > 24.9) & (df_dev["BMI"] <= 29.9), "NewBMI"] = NewBMI[2]
df_dev.loc[(df_dev["BMI"] > 29.9) & (df_dev["BMI"] <= 34.9), "NewBMI"] = NewBMI[3]
df_dev.loc[(df_dev["BMI"] > 34.9) & (df_dev["BMI"] <= 39.9), "NewBMI"] = NewBMI[4]
df_dev.loc[df_dev["BMI"] > 39.9 ,"NewBMI"] = NewBMI[5]

In [ ]:
df_dev.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI
0,1,144.0,82.0,40.0,1.0,41.3,0.607,28,0,Obesity 3
1,0,179.0,50.0,36.0,159.0,37.8,0.455,22,1,Obesity 2
2,9,184.0,85.0,15.0,1.0,30.0,1.213,49,1,Obesity 1
3,0,134.0,58.0,20.0,291.0,26.4,0.352,21,0,Overweight
4,4,142.0,86.0,1.0,1.0,44.0,0.645,22,1,Obesity 3


In [ ]:
# A categorical variable creation process is performed according to the insulin value.
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"

In [ ]:
# The operation performed was added to the dataframe.
df_dev = df_dev.assign(NewInsulinScore=df_dev.apply(set_insulin, axis=1))

In [ ]:
df_dev.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI,NewInsulinScore
0,1,144.0,82.0,40.0,1.0,41.3,0.607,28,0,Obesity 3,Abnormal
1,0,179.0,50.0,36.0,159.0,37.8,0.455,22,1,Obesity 2,Normal
2,9,184.0,85.0,15.0,1.0,30.0,1.213,49,1,Obesity 1,Abnormal
3,0,134.0,58.0,20.0,291.0,26.4,0.352,21,0,Overweight,Abnormal
4,4,142.0,86.0,1.0,1.0,44.0,0.645,22,1,Obesity 3,Abnormal


In [ ]:
# Some intervals were determined according to the glucose variable and these were assigned categorical variables.
NewGlucose = pd.Series(["Low", "Normal", "Overweight", "Secret", "High"], dtype = "category")
df_dev["NewGlucose"] = NewGlucose
df_dev.loc[df_dev["Glucose"] <= 70, "NewGlucose"] = NewGlucose[0]
df_dev.loc[(df_dev["Glucose"] > 70) & (df_dev["Glucose"] <= 99), "NewGlucose"] = NewGlucose[1]
df_dev.loc[(df_dev["Glucose"] > 99) & (df_dev["Glucose"] <= 126), "NewGlucose"] = NewGlucose[2]
df_dev.loc[df_dev["Glucose"] > 126 ,"NewGlucose"] = NewGlucose[3]

In [ ]:
df_dev.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI,NewInsulinScore,NewGlucose
0,1,144.0,82.0,40.0,1.0,41.3,0.607,28,0,Obesity 3,Abnormal,Secret
1,0,179.0,50.0,36.0,159.0,37.8,0.455,22,1,Obesity 2,Normal,Secret
2,9,184.0,85.0,15.0,1.0,30.0,1.213,49,1,Obesity 1,Abnormal,Secret
3,0,134.0,58.0,20.0,291.0,26.4,0.352,21,0,Overweight,Abnormal,Secret
4,4,142.0,86.0,1.0,1.0,44.0,0.645,22,1,Obesity 3,Abnormal,Secret


One hot encoding cho các biến mới tạo ở trên

tách riêng ra là được, t đang để chung 1 chỗ

In [ ]:
# Here, by making One Hot Encoding transformation, categorical variables were converted into numerical values. It is also protected from the Dummy variable trap.
df_train = pd.get_dummies(df_train, columns =["NewBMI","NewInsulinScore", "NewGlucose"], drop_first = True)
df_dev = pd.get_dummies(df_dev, columns =["NewBMI","NewInsulinScore", "NewGlucose"], drop_first = True)

In [ ]:
df_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,NewBMI_Obesity 1,NewBMI_Obesity 2,NewBMI_Obesity 3,NewBMI_Overweight,NewBMI_Underweight,NewInsulinScore_Normal,NewGlucose_Low,NewGlucose_Normal,NewGlucose_Overweight,NewGlucose_Secret
0,1,80.0,55.0,1.0,0.0,19.1,0.258,21,0,0,0,0,0,0,0,0,1,0,0
1,1,124.0,74.0,36.0,0.0,27.8,0.100,30,0,0,0,0,1,0,0,0,0,1,0
2,10,108.0,66.0,1.0,0.0,32.4,0.272,42,1,1,0,0,0,0,0,0,0,1,0
3,2,120.0,76.0,37.0,105.0,39.7,0.215,29,0,0,1,0,0,0,1,0,0,1,0
4,11,136.0,84.0,35.0,130.0,28.3,0.260,42,1,0,0,0,1,0,1,0,0,0,1


In [ ]:
categorical_df_train = df_train[['NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret']]
categorical_df_train.head()

,NewBMI_Obesity 1,NewBMI_Obesity 2,NewBMI_Obesity 3,NewBMI_Overweight,NewBMI_Underweight,NewInsulinScore_Normal,NewGlucose_Low,NewGlucose_Normal,NewGlucose_Overweight,NewGlucose_Secret
0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,1,0
3,0,1,0,0,0,1,0,0,1,0
4,0,0,0,1,0,1,0,0,0,1


In [ ]:
categorical_df_dev = df_dev[['NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret']]
categorical_df_dev.head()

,NewBMI_Obesity 1,NewBMI_Obesity 2,NewBMI_Obesity 3,NewBMI_Overweight,NewBMI_Underweight,NewInsulinScore_Normal,NewGlucose_Low,NewGlucose_Normal,NewGlucose_Overweight,NewGlucose_Secret
0,0,0,1,0,0,0,0,0,0,1
1,0,1,0,0,0,1,0,0,0,1
2,1,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,1


In [ ]:
y_train = df_train["Outcome"]
X_train = df_train.drop(["Outcome",'NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret'], axis = 1)
cols_train = X_train.columns
index_train = X_train.index

y_dev = df_dev["Outcome"]
X_dev = df_dev.drop(["Outcome",'NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret'], axis = 1)
cols_dev = X_dev.columns
index_dev = X_dev.index

In [ ]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1,80.0,55.0,1.0,0.0,19.1,0.258,21
1,1,124.0,74.0,36.0,0.0,27.8,0.100,30
2,10,108.0,66.0,1.0,0.0,32.4,0.272,42
3,2,120.0,76.0,37.0,105.0,39.7,0.215,29
4,11,136.0,84.0,35.0,130.0,28.3,0.260,42


chuẩn hóa train và dev

In [ ]:
# Normalization có thể dùng RobustScaler, MinMax..
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_dev = scaler.transform(X_dev)

X_train = pd.DataFrame(X_train, columns = cols_train, index = index_train)
X_dev = pd.DataFrame(X_dev, columns = cols_dev, index = index_dev)

In [ ]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,-0.860348,-1.343477,-0.678247,-1.272899,-0.714864,-1.678137,-0.675097,-1.052800
1,-0.860348,0.117433,0.292153,1.005056,-0.714864,-0.531398,-1.169962,-0.281308
2,1.780605,-0.413807,-0.116437,-1.272899,-0.714864,0.074924,-0.631248,0.747347
3,-0.566909,-0.015377,0.394301,1.070140,0.275215,1.037131,-0.809776,-0.367030
4,2.074044,0.515863,0.802890,0.939972,0.510949,-0.465493,-0.668833,0.747347


In [ ]:
X_dev.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,-0.860348,0.781483,0.700743,1.265394,-0.705435,1.248026,0.417990,-0.452751
1,-1.153787,1.943570,-0.933616,1.005056,0.784399,0.786694,-0.058082,-0.967079
2,1.487165,2.109583,0.853964,-0.361717,-0.705435,-0.241418,2.316016,1.347396
3,-1.153787,0.449458,-0.525026,-0.036295,2.029071,-0.715931,-0.380684,-1.052800
4,0.019969,0.715078,0.905038,-1.272899,-0.705435,1.603911,0.537008,-0.967079


nối feature ban đầu đã đc chuẩn hóa với feature đc encoding

In [ ]:
X_train = pd.concat([X_train,categorical_df_train], axis = 1)
X_dev = pd.concat([X_dev,categorical_df_dev], axis = 1)

Model Adaboost ra 0.84199

In [ ]:
abc_tuned = AdaBoostClassifier(**{'algorithm': 'SAMME.R', 'learning_rate': 1.0, 'n_estimators': 10})
abc_tuned = abc_tuned.fit(X_train,y_train)
y_pred = abc_tuned.predict(X_dev)
f1 = f1_score(y_dev,y_pred, average='macro')
print(f1)

0.8419972640218878
